# Advanced Machine Learning - Assignment 1

Welcome to your first programming assignment. You will build a classifier based on logistic regression to recognize cats. This assignment will teach you how to do this and hone your intuitions about the course material.

**Instructions:**
- Do not use loops (for/while) in your code unless the instructions explicitly ask you to do so.

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Initializing parameters
    - Calculating the cost function and its gradient
    - Using the gradient descent algorithm
- Gather all three above functions into the main model function correctly.

## 1 - Packages ##

First, run the cell below to import all the necessary packages during this assignment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy

## 2 - Overview of the problem set ##

**Problem Statement**: You are given a dataset containing:
- a training set of `m_train` images labeled as cat ($y=1$) or non-cat ($y=0$)
- a test set of `m_test` images labeled as cat or non-cat
- each image is of shape (`num_px`, `num_px`, 3), where 3 is for the three channels (RGB). Thus, each image is square (height = `num_px`) and (width = `num_px`).

You will build a simple image-recognition algorithm to classify pictures as cat or non-cat correctly.

Get familiar with the dataset. Load the data by running the following code.

In [ ]:
train_dataset = h5py.File('train_catvnoncat.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

test_dataset = h5py.File('test_catvnoncat.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels

classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

We added "_orig" at the end of image datasets (train and test) because we will preprocess them. After preprocessing, we will end up with `train_set_x` and `test_set_x` (the labels `train_set_y` and `test_set_y` do not need any preprocessing).

Each line of your `train_set_x_orig` and `test_set_x_orig` is an array representing an image. You can visualize an example by running the following code. Feel free to change the `index` value and re-run to see other images. 

In [ ]:
# Example of a picture
index = 25
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

Many software bugs come from having matrix/vector dimensions that do not match. If you can keep your matrix/vector dimensions straight, you will go a long way toward eliminating many bugs. 

**Exercise:** Find the values for:
- `m_train` (number of training examples)
- `m_test` (number of test examples)
- `num_px` (= height = width of a training image)

Remember that `train_set_x_orig` is a numpy array of shape (m_train, num_px, num_px, 3). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:
# YOUR CODE HERE

# YOUR CODE ENDS HERE

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**Expected Output for m_train, m_test and num_px**: 
<table style="width:15%">
  <tr>
    <td>m_train</td>
    <td> 209 </td> 
  </tr>
  
  <tr>
    <td>m_test</td>
    <td> 50 </td> 
  </tr>
  
  <tr>
    <td>num_px</td>
    <td> 64 </td> 
  </tr>
  
</table>

For convenience, you should now reshape images of shape (`num_px`, `num_px`, 3) in a numpy array of shape (`num_px` $*$ `num_px` $*$ 3, 1). After this, our training (and test) dataset is a numpy array where each column represents a flattened image. There should be `m_train` (respectively `m_test`) columns.

**Exercise:** Reshape the training and test data sets so that images of size (`num_px`, `num_px`, 3) are flattened into single vectors of shape (`num_px` $*$ `num_px` $*$ 3, 1).

A trick when you want to flatten a matrix $X$ of shape ($a$, $b$, $c$, $d$) to a matrix $X$_flatten of shape ($b * c * d$, $a$) is to use: 
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X
```


In [ ]:
# Reshape the training and test examples

# YOUR CODE HERE

# YOUR CODE ENDS HERE

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

**Expected Output**: 

<table style="width:35%">
  <tr>
    <td>train_set_x_flatten shape</td>
    <td> (12288, 209)</td> 
  </tr>
  <tr>
    <td>train_set_y shape</td>
    <td>(1, 209)</td> 
  </tr>
  <tr>
    <td>test_set_x_flatten shape</td>
    <td>(12288, 50)</td> 
  </tr>
  <tr>
    <td>test_set_y shape</td>
    <td>(1, 50)</td> 
  </tr>
  <tr>
  <td>sanity check after reshaping</td>
  <td>[17 31 56 22 33]</td> 
  </tr>
</table>

To represent color images, the red, green, and blue channels (RGB) must be specified for each pixel, so the pixel value is a vector of three numbers ranging from 0 to 255.

One common preprocessing step in machine learning is to center and standardize your dataset. You subtract the mean of the whole numpy array from each example and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to divide every row of the dataset by 255 (the maximum value of a pixel channel).

Let us standardize our dataset.

In [ ]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

## 3 - General Architecture of the learning algorithm ##

It is time to design a simple algorithm to distinguish cat and non-cat images. You will build a Logistic Regression model. 

**Mathematical expression of the algorithm**:

For one example $x_{i}$:
$$z_{i} = w^T x_{i} + b \tag{1}$$
$$\hat{y}_{i} = a_{i} = sigmoid(z_{i})\tag{2}$$ 
$$L(a_{i}, y_{i}) =  - y_{i}  \log(a_{i}) - (1-y_{i} )  \log(1-a_{i})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m L(a_{(i)}, y_{(i)})\tag{6}$$

**Key steps**:
In this exercise, you will carry out the following steps: 
- Initialize the parameters of the model
- Learn the parameters for the model by minimizing the cost
- Use the learned parameters to make predictions (on the test set)
- Analyse the results and conclude

## 4 - Building the parts of the algorithm ## 

The main steps for building the model are:
1. Define the model structure (such as the number of input features) 
2. Initialize the model's parameters
3. Loop:
    - Calculate current cost/loss
    - Calculate the current gradient
    - Update parameters (gradient descent)

You often build 1-3 separately and integrate them into one function we call `model()`.

### 4.1 - Helper functions

**Exercise**: Using your code from "Python Basics", implement `sigmoid()`. You need to compute $sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$ to make predictions. Use `np.exp()`.

In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE
    
    return s

In [ ]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))


**Expected Output**: 

<table>
  <tr>
    <td>sigmoid([0, 2])</td>
    <td> [ 0.5         0.88079708]</td> 
  </tr>
</table>

### 4.2 - Initializing parameters

**Exercise:** Implement parameter initialization in the cell below. You have to initialize w as a vector of zeros. If you do not know what numpy function to use, look up `np.zeros()` in the Numpy library's documentation.

In [ ]:
# GRADED FUNCTION: initialize_with_zeros

def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))
print(initialize_with_zeros(2)[0])


**Expected Output**: 


<table style="width:15%">
    <tr>
        <td>   w   </td>
        <td> [[ 0.]
 [ 0.]] </td>
    </tr>
    <tr>
        <td>   b   </td>
        <td> 0 </td>
    </tr>
</table>

For image inputs, $w$ will be of shape (`num_px` $\times$ `num_px` $\times$ 3, 1).

### 4.3 - Calculate the cost function

Now that your parameters are initialized, you can do the steps for learning the parameters.

**Exercise:** Implement a function `propagate()` that computes the cost function and its gradient.

**Hints**:

Forward Propagation:
- You get $X$
- You compute $A = \sigma(w^T X + b) = (a_{0}, a_{1}, ..., a_{m-1}, a_{m})$
- You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y_{i}\log(a_{i})+(1-y_{i})\log(1-a_{i})$

Here are the two formulas you will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a_{i}-y_{i})\tag{8}$$

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))


**Expected Output**:

<table style="width:50%">
    <tr>
        <td>   dw   </td>
      <td> [[ 0.99845601]
     [ 2.39507239]]</td>
    </tr>
    <tr>
        <td>   db   </td>
        <td> 0.00145557813678 </td>
    </tr>
    <tr>
        <td>   cost   </td>
        <td> 5.801545319394553 </td>
    </tr>

</table>

### d) Optimization
- You have initialized your parameters
- You are also able to compute a cost function and its gradient
- Now, you want to update the parameters using gradient descent

**Exercise:** Write down the optimization function. The goal is to learn $w$ and $b$ by minimizing the cost function $J$. For a parameter $\theta$, the update rule is $ \theta = \theta - \alpha \text{ } d\theta$, where $\alpha$ is the learning rate.

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        # YOUR CODE HERE
        
        # YOUR CODE ENDS HERE
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        # YOUR CODE HERE
        
        # YOUR CODE ENDS HERE
      
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))


**Expected Output**: 

<table style="width:40%">
    <tr>
       <td> w </td>
       <td>[[ 0.19033591][ 0.12259159]] </td>
    </tr> 
    <tr>
       <td> b </td>
       <td> 1.92535983008 </td>
    </tr>  
    <tr>
       <td> dw </td>
       <td> [[ 0.67752042] [ 1.41625495]] </td>
    </tr>
    <tr>
       <td> db </td>
       <td> 0.219194504541 </td>
    </tr>

</table>

**Exercise:** The previous function will output the learned $w$ and $b$. We can use $w$ and $b$ to predict the labels for a dataset $X$. Implement the `predict()` function. There are two steps to computing predictions:

1. Calculate $\hat{Y} = A = \sigma(w^T X + b)$

2. Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5), and stores the predictions in a vector `Y_prediction`. You can use an `if`/`else` statement in a `for` loop (though there is also a way to vectorize this). 

In [ ]:
# GRADED FUNCTION: predict

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE

    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        # YOUR CODE HERE
        
        # YOUR CODE ENDS HERE
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [ ]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))


**Expected Output**: 

<table style="width:30%">
    <tr>
         <td>
             predictions
         </td>
          <td>
            [[ 1.  1.  0.]]
         </td>  
   </tr>

</table>

<font color='blue'>
What to remember:
    
You have implemented several functions that:
- Initialize ($w$, $b$)
- Optimize the loss iteratively to learn parameters ($w$, $b$):
    - computing the cost and its gradient 
    - updating the parameters using gradient descent
- Use the learned ($w$, $b$) to predict the labels for a given set of examples

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    # YOUR CODE HERE
    
    # YOUR CODE ENDS HERE

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

Run the following cell to train your model.

In [ ]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)


**Expected Output**: 

<table style="width:40%"> 
    <tr>
        <td>Cost after iteration 0:</td>
        <td>0.693147</td>
    </tr>
    <tr>
        <td> <center> $\vdots$ </center> </td> 
        <td> <center> $\vdots$ </center> </td> 
    </tr>  
    <tr>
        <td> Train Accuracy  </td> 
        <td> 99.04306220095694 % </td>
    </tr>
    <tr>
        <td>Test Accuracy </td> 
        <td> 70.0 % </td>
    </tr>
</table> 

**Comment**: Training accuracy is close to 100%. This is a good sanity check: your model works and has enough capacity to fit the training data. Test error is around 68%. It is not bad for this simple model, given the small dataset we used and that logistic regression is a linear classifier. But no worries, you will build an even better classifier in the following programming assignment!

Also, you see that the model is overfitting the training data. Later, you will reduce overfitting by using regularization. Using the code below (and changing the `index` variable), you can look at predictions on pictures of the test set.

In [ ]:
index = 1
plt.imshow(test_set_x[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0,index]) + ", you predicted that it is a \"" + classes[int(d["Y_prediction_test"][0,index])].decode("utf-8") +  "\" picture.")

Let us also plot the cost function and the gradients.

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

**Interpretation**:
You can see the cost decreasing. It shows that the parameters are being learned. However, you see that you could train the model even more on the training set. Try to increase the number of iterations in the cell above and rerun the cells. You might see that the training set accuracy goes up, but the test set accuracy goes down. This is called overfitting. 